In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('Users')

+---------+-----------+-------------------------+
| user_id |    name   |           mail          |
+---------+-----------+-------------------------+
|    1    |  Winston  |   winston@leetcode.com  |
|    2    |  Jonathan |     jonathanisgreat     |
|    3    | Annabelle |   bella-@leetcode.com   |
|    4    |   Sally   | sally.come@leetcode.com |
|    5    |   Marwan  | quarz#2020@leetcode.com |
|    6    |   David   |    david69@gmail.com    |
|    7    |  Shapiro  |   .shapo@leetcode.com   |
+---------+-----------+-------------------------+


### Find the users who have valid emails.

#### *A valid e-mail has a prefix name and a domain where:*

* The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.', and/or dash '-'. 
* The prefix name must start with a letter.

#### *The domain is '@leetcode.com'.*
```
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
+---------+-----------+-------------------------+
Explanation: 
The mail of user 2 does not have a domain.
The mail of user 5 has the # sign which is not allowed.
The mail of user 6 does not have the leetcode domain.
The mail of user 7 starts with a period.
```

# Using `regex`

In [3]:
%%sql 

SELECT
    mail
FROM Users
WHERE mail REGEXP '^[a-zA-Z]'


mail
winston@leetcode.com
jonathanisgreat
bella-@leetcode.com
sally.come@leetcode.com
quarz#2020@leetcode.com
david69@gmail.com


In [4]:
%%sql 

SELECT
    user_id,
    name,
    mail
FROM Users
WHERE mail REGEXP '^[a-zA-Z][a-zA-Z0-9\.\_\-]*@leetcode\.com$'


user_id,name,mail
1,Winston,winston@leetcode.com
3,Annabelle,bella-@leetcode.com
4,Sally,sally.come@leetcode.com


In [5]:
SELECT
    mail
FROM Users
WHERE mail REGEXP '^[a-zA-Z][a-zA-Z0-9\.\_\-]*@leetcode\.com$'

IndentationError: unexpected indent (1781598578.py, line 2)

## Using `SUBSTRING_INDEX`

In [ ]:
%%sql

SELECT 
    mail
FROM Users
WHERE mail LIKE "%@leetcode.com"

In [ ]:
%%sql

SELECT 
    SUBSTRING_INDEX(mail, '@', 1)
FROM Users
WHERE mail LIKE "%@leetcode.com"

In [ ]:
%%sql

SELECT 
    user_id,
    name,
    mail
FROM Users
WHERE mail LIKE "%@leetcode.com" 
    AND SUBSTRING_INDEX(mail, '@', 1) NOT LIKE '%#%'
    AND SUBSTRING_INDEX(mail, '@', 1) NOT LIKE '.%'

# Using Pandas

In [ ]:
users_query = %sql SELECT * FROM Users # type: ignore
users_df = users_query.DataFrame()

display(users_df)

In [ ]:
filtered_df = users_df[users_df['mail'].str.contains('@leetcode.com')]
filtered_df

In [ ]:
filtered_df['mail'].str.split('@')

In [ ]:
filtered_df['mail'].str.split('@')

In [ ]:
filtered_df['mail'].str.split('@').str.get(0)

In [ ]:
filtered_df['email_local_part'] = filtered_df['mail'].str.split('@').str.get(0)

filtered_df

In [ ]:
# Filter out those that start with a letter
pattern = r'^[a-zA-Z]'
filtered_df['email_local_part'].str.contains(pattern, regex=True)

In [ ]:
#  Matches zero or more occurrences of any uppercase or lowercase letter, digit, underscore, period, or dash.
pattern = r'^[a-zA-Z][a-zA-Z0-9_.-]*'
filtered_df['email_local_part'].str.contains(pattern, regex=True)

### Without the `$` end-of-string(EOS) anchor, the pattern only requires the local part to start with a letter and followed by zero or more alphanumeric characters, underscores, periods, or dashes. It doesn't account for any additional characters after the valid local part, so the regex matches only the art that satisfies the pattern and ignores the rest of the email.

In [ ]:
# Ensure that the pattern matches the entire string from start to end, leaving no room for additional
# characters like '#' after the valid part.
pattern = r'^[a-zA-Z][a-zA-Z0-9_.-]*$'
filtered_df['email_local_part'].str.contains(pattern, regex=True)

In [ ]:
mask = filtered_df['email_local_part'].str.contains(pattern, regex=True)
final_df = filtered_df[mask]
final_df

In [ ]:
final_df.drop('email_local_part', axis=1, inplace=True)
final_df